# 中国最古の円周率の求め方

3世紀前半から中盤ごろ、劉徽という人物は内接正多角形の面積と、その各辺に長方形をつけた図形の面積と円の面積を比較することで、円周率を計算しました。以下の図のように、黄色い内接正多角形に、その外側赤いの部分と緑の部分を合わせた長方形をつけると、内接正多角形は円の面積より小さく、長方形をくっつけると円の面積より大きくなります。

In [23]:
from IPython.display import HTML
HTML('<img src="https://upload.wikimedia.org/wikipedia/commons/3/34/Liuhui_Pi_Inequality.svg" alt="wikimediaから引用"/>')

円の半径を $1$ とし、内接正 $n$ 角形の面積を $A_n$ とおきます。このとき、上の図の黄色い多角形が内接正 $n$ 角形だとすると、黄色い多角形と緑の部分を合わせたものは内接正 $2n$ 角形で、赤い部分と緑の部分の面積は等しいので、黄、赤、緑の部分を合わせた面積は

$$A_{2n} + (A_{2n} -A_n) = 2A_{2n} -A_n$$

となります。円の面積は $\pi r^2 = \pi$ なので

$$A_{2n} < \pi < 2A_{2n} -A_n$$

という不等式が成り立ちます。よって円の内接正 $A_n$ 角形の面積を求められれば、円周率を求めることができます。

In [6]:
from IPython.display import HTML
HTML('<img src="https://upload.wikimedia.org/wikipedia/commons/4/46/Liuhui_geyuanshu.svg"/>')

内接 $n$ 角形の1辺の長さを $a_n$ とおくと、上の赤と緑の三角形を合わせた三角形は底辺 $1$, 高さ $\frac{a_n}{2}$ で、内接正 $2n$ 角形はこれを $2n$ 個集めたものなので、$A_{2n}$ の値は

$$\begin{align}
& A_{2n} = 2n \cdot \frac{a_n}{4} = \frac{n a_n}{4}
\end{align}$$

となります。よって $a_n$ が分かれば $A_{2n}$ が求まります。

まず辺 $OP$ の長さは

$$OP = \sqrt{r^2 -\left(\frac{a_n}{2}\right)^2} = \frac{1}{2}\sqrt{4 -a_n^2}$$

で与えられます。$a_{2n}$ の長さは

$$\begin{align*}
a_{2n}^2 &= \left(\frac{a_n}{2}\right)^2 +(1 -OP)^2 = \frac{1}{4} a_n^2 + 1 -2 OP + OP^2 \\
&= \frac{1}{4} a_n^2 + 1 -\sqrt{4-a_n^2} + 1 -\frac{1}{4} a_n^2 \\
&= 2 -\sqrt{4 -a_n^2}
\end{align*}$$

と $a_n$ を用いて表されます。これで、内接正多角形の1辺の長さを求めれば、円周率を求めることができます。

アルキメデスの求め方では外接多角形と内接多角形の両方の辺の長さを求めなければならなかったことと比べると、劉徽の方法は内接多角形の辺だけ求めればよいので、必要な平方根の計算がほぼ半分になり、て計算においては効率が良いです。

それでは実際に計算してみましょう。劉徽は正 $6$ 角形から始めて $192 = 6 \cdot 2^5$ 角形まで計算したようなので、今回も正 $6$ 角形から始めます。

In [1]:
import math
from functools import lru_cache


@lru_cache
def square_length_inscribed_polygon(e: int) -> float:
    """半径1の円の内接正6×2^n角形の1辺の長さの2乗を計算する"""
    if e < 0:
        raise ValueError(f"Input value of e is {e}. e must be more than 0.")

    if e == 0:
        return 1

    return 2 - math.sqrt(4 - square_length_inscribed_polygon(e-1))


@lru_cache
def area_inscribed_polygon(e: int) -> float:
    """半径1の円の内接正6×2^n角形の面積を計算する"""
    if e < 0:
        raise ValueError(f"Input value of e is {e}. e must be more than 0.")

    square_length_one_side = square_length_inscribed_polygon(e)
    
    return (6 * 2 ** e) \
        * math.sqrt(square_length_one_side) \
        * math.sqrt(4 -square_length_one_side) \
        / 4    

In [40]:
import sys
sys.path.append('/home/jovyan/work/')
from lib.utils import color_pi, PI_50

A_prev = area_inscribed_polygon(0)
for e in range(1, 10):
    An = area_inscribed_polygon(e)
    
    print(
        f"n = {str(6 * 2 ** e):<4}: " + f"{color_pi(str(An)[:10], PI_50)}"
        f" < π < {color_pi(str(2 * An - A_prev)[:10], PI_50)}"
    )
    A_prev = An

n = 12  : 3.00000000 < π < 3.40192378
n = 24  : 3.10582854 < π < 3.21165708
n = 48  : 3.13262861 < π < 3.15942868
n = 96  : 3.13935020 < π < 3.14607179
n = 192 : 3.14103195 < π < 3.14271369
n = 384 : 3.14145247 < π < 3.14187299
n = 768 : 3.14155760 < π < 3.14166274
n = 1536: 3.14158389 < π < 3.14161017
n = 3072: 3.14159046 < π < 3.14159703


劉徽は $n = 192$ まで計算し

$$\begin{align*}
& 314 + \frac{64}{625} < 100 \pi < 314 + \frac{169}{625} \\
\Leftrightarrow \ & 3.14 + \frac{64}{62500} < \pi < 3.14 + \frac{169}{62500}
\end{align*}$$

つまり

$$3.141024 < \pi < 3.142704$$

を得ました。上記の計算結果の $n = 192$ と比べると、上からの評価が若干良くなっていますが、これは平方根の計算時にある桁以降を切り捨てていたため $A_n$ の値が実際の値より小さく、$2A_{2n} -A_n$ も実際の値より小さくなったためだと思われます。
正確な評価には $A_n$ を上下から評価し、上からの評価には大きめの値を用いる必要がありますが、そこまではしなかったようです。

In [22]:
print(f"3.14+64/62500 = {3.14 + 64 / 62500}")
print(f"3.14+169/62500 = {3.14 + 169 / 62500}")

3.14+64/62500 = 3.1410240000000003
3.14+169/62500 = 3.142704


## アルキメデスの方法との比較

劉徽の方法とアルキメデスの方法と比較すると、平方根の計算が半分になっている反面、

# 無限積公式

$c_n = \frac{A_n}{A_{2n}}$ とおくと、

$$\begin{align*}
c_{2n} &= \sqrt{\frac{1}{2} + \frac{1}{2} c_n} \\
\frac{2A_{4n} -A_{2n}}{2A_{2n} -A_n} &= \frac{2 -c_{2n}}{c_{2n}(2 -c_n)} \\
\end{align*}$$

が成り立ちます。$m \geq 2$ に対して

$$\begin{align}
2A_{2^{m+2}} -A_{2^{m+1}} &= (2A_8 -A_4)\prod_{k=2}^m \frac{2A_{2^{k+2}} -A_{2^{k+1}} }{2A_{2^{k+1}} -A_2^k} \\
&= A_8 (2 -c_4) \frac{(2 -c_{2^{m+1}})}{2 -c_4} \prod_{k=2}^m \frac{1}{c_{2^{m+1}}} \\
&= A_4 (2 -c_{2^{m+1}}) \prod_{k=2}^m \frac{1}{c_{2^m}}
\end{align}$$

なので

$$\frac{\pi}{2} < (2 -c_{2^{m+1}}) \prod_{k=2}^m \frac{1}{c_{2^m}}$$

となります。$c_{2^{m+1}} \to 1$, $(m \to \infty)$ なので

$$\begin{align}
& \frac{2}{\pi} = \prod_{k =2}^{\infty} c_{2^k} \\
= \ & \sqrt{\frac{1}{2}} \sqrt{\frac{1}{2} + \frac{1}{2} \sqrt{\frac{1}{2}}} \sqrt{\frac{1}{2} + \frac{1}{2}\sqrt{\frac{1}{2} + \frac{1}{2} \sqrt{\frac{1}{2}}}} \cdots
\end{align}$$

となり、ヴィエトの公式が得られます。

ヴィエトの公式は円周率を下からしか評価できませんでしたが、劉徽の式を用いた無限積の公式は $(2 -c_{2^{m+1}})$ がかかっているおかげで円周率を上から評価できます。

In [26]:
from typing import Generator, Union
import math

def viete_seq() -> Generator[Union[float, int], None, None]:
    init = math.sqrt(1 / 2)
    n = 1
    a = init
    while True:
        yield (a, n)
        a = math.sqrt((1 + a ) / 2)
        n = n + 1

In [31]:
vseq = viete_seq()
v_0, _ = next(vseq)
v_prod = 1

for _ in range(0, 10):
    (v_n, n) = next(vseq)
    v_prod = v_prod * v_n
    pi = 2 * math.sqrt(2) * (2 -v_n) / v_prod
    print(f"n = {n}: {pi}")

n = 2: 3.2945077930952467
n = 3: 3.181422845595387
n = 4: 3.1516518288338267
n = 5: 3.1441138233635666
n = 6: 3.1422233449107932
n = 7: 3.141750351355829
n = 8: 3.1416320795898818
n = 9: 3.1416025101872282
n = 10: 3.1415951177452404
n = 11: 3.1415932696290354
